In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

In [ ]:
peft_model_path = 'output/chatGLM3_6B_QLoRA_t32/checkpoint-400'

config = PeftConfig.from_pretrained(peft_model_path)
q_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_compute_dtype=torch.float32)

base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                        quantization_config=q_config,
                                        trust_remote_code=True,
                                        device_map='auto')

In [ ]:
input_text = '你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我想知道国金核心资产A的基金经理是谁，以及他的年化回报率和管理的总规模 \n query中提到的产品标准名可能是：国金核心资产一年持有期混合型证券投资基金C类、国金核心资产一年持有期混合型证券投资基金A类、富国大盘核心资产混合型证券投资基金、国联安核心资产策略混合型证券投资基金、创金合信核心资产混合型证券投资基金C类、创金合信核心资产混合型证券投资基金A类、金鹰核心资源混合型证券投资基金C类、金鹰核心资源混合型证券投资基金A类、金元顺安核心动力混合型证券投资基金、汇安核心资产混合型证券投资基金C类、汇安核心资产混合型证券投资基金A类、富国核心趋势混合型证券投资基金C类、富国核心趋势混合型证券投资基金A类、国投瑞银核心企业混合型证券投资基金、华夏核心资产混合型证券投资基金C类、华夏核心资产混合型证券投资基金A类、国联安核心优势混合型证券投资基金A类、民生加银核心资产股票型证券投资基金C类、民生加银核心资产股票型证券投资基金A类、国联核心成长灵活配置混合型证券投资基金、博时核心资产精选混合型证券投资基金C类、博时核心资产精选混合型证券投资基金A类、创金合信量化核心混合型证券投资基金C类、创金合信量化核心混合型证券投资基金A类、创金合信核心价值混合型证券投资基金C类、创金合信核心价值混合型证券投资基金A类、鑫元核心资产股票型发起式证券投资基金C类、鑫元核心资产股票型发起式证券投资基金A类、海富通消费核心资产混合型证券投资基金C类、海富通消费核心资产混合型证券投资基金A类、富国核心优势混合型发起式证券投资基金C类、富国核心优势混合型发起式证券投资基金A类、光大保德信核心资产混合型证券投资基金C类、光大保德信核心资产混合型证券投资基金A类、交银施罗德核心资产混合型证券投资基金C类、交银施罗德核心资产混合型证券投资基金A类、招商资管核心优势混合型集合资产管理计划C类、招商资管核心优势混合型集合资产管理计划A类、国寿安保核心产业灵活配置混合型证券投资基金、金信核心竞争力灵活配置混合型证券投资基金A类、汇添富大盘核心资产增长混合型证券投资基金C类、汇添富大盘核心资产增长混合型证券投资基金A类、国泰核心价值两年持有期股票型证券投资基金C类、国泰核心价值两年持有期股票型证券投资基金A类、国泰金鹿混合型证券投资基金、国联安核心趋势一年持有期混合型证券投资基金C类、国联安核心趋势一年持有期混合型证券投资基金A类、中金安心回报灵活配置混合型集合资产管理计划C类、中金安心回报灵活配置混合型集合资产管理计划A类、银华全球核心优选证券投资基金'
print(f'输入：\n{input_text}')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
response, history = base_model.chat(tokenizer=tokenizer, query=input_text)
print(f'微调前：\n{response}')

In [ ]:
model = PeftModel.from_pretrained(base_model, peft_model_path)
response, history = model.chat(tokenizer=tokenizer, query=input_text)
print(f'微调后: \n{response}')

In [5]:
import json

def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

In [6]:
import pandas as pd

json_file_path = 'data/dev.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,input,output
0,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
1,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
2,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
3,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
4,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."


In [7]:
from colorama import Fore, Style

def highlight_diff(str1, str2):
    result = ''
    for char1, char2 in zip(str1, str2):
        if char1 != char2:
            result += Fore.RED + char1 + Style.RESET_ALL
        else:
            result += char1
    # 处理长度不一致的情况
    if len(str1) > len(str2):
        result += Fore.RED + str1[len(str2):] + Style.RESET_ALL
    elif len(str2) > len(str1):
        result += Fore.RED + str2[len(str1):] + Style.RESET_ALL
    return result

str1 = "example string"
str2 = "esample string"
print(highlight_diff(str1, str2))

example string


In [ ]:
total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    input = row["input"]
    output = row["output"]
    response, history = model.chat(tokenizer=tokenizer, query=input)
    
    total_eval_count += 1
    if response == output:
        correct_count += 1
    else:
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, response))
        print(response)
        print()
    
    if total_eval_count == 10:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

In [1]:
from transformers import AutoTokenizer, AutoModel

base_line_model = "../ChatGLM2-6B/ptuning/output/checkpoint-100"

base_tokenizer = AutoTokenizer.from_pretrained(base_line_model, trust_remote_code=True)
base_model = AutoModel.from_pretrained(base_line_model, trust_remote_code=True).half().cuda()
base_model = base_model.eval()

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 3/3 [02:09<00:00, 43.31s/it]


In [2]:
response, history = base_model.chat(base_tokenizer, "你好", history=[])
print(response)

{"relevant APIs": [{"api_id": "0", "api_name": "查询代码", "required_parameters": [["华能国际"]], "rely_apis": [], "tool_name": "股票查询"}, {"api_id": "1", "api_name": "查询收盘价", "required_parameters": ["api_0的结果", "今日"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "2", "api_name": "查询收盘价", "required_parameters": ["api_0的结果", "今日"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "3", "api_name": "减法计算", "required_parameters": ["api_2的结果", "api_1的结果"], "rely_apis": ["1", "2"], "tool_name": "数值计算"}, {"api_id": "4", "api_name": "乘法计算", "required_parameters": ["api_3的结果", "100"], "rely_apis": ["3"], "tool_name": "数值计算"}], "result": ["api_4的结果"]}


In [9]:
from tools.standard_name_utils import optimize_parameters

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    input = row["input"]
    output = row["output"]
    
    response, history = base_model.chat(tokenizer=base_tokenizer, query=input)
    
    optimized_resp = optimize_parameters(response, input)
    
    total_eval_count += 1
    if optimized_resp == output:
        correct_count += 1
    # else:
        # print("-----query input-----")
        # print(input)
        # print("-----output diff-----")
        # print(highlight_diff(output, response))
        # print(response)
        # print()
    
    if total_eval_count % 10 == 0:
        print("现在是第" + f"{total_eval_count}" + "条数据，总预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")
        
    if total_eval_count == 100:
        break

现在是第10.0条数据，总预测正确的比例：60.00%
现在是第20.0条数据，总预测正确的比例：45.00%
现在是第30.0条数据，总预测正确的比例：56.67%
现在是第40.0条数据，总预测正确的比例：57.50%
现在是第50.0条数据，总预测正确的比例：58.00%


KeyboardInterrupt: 

In [10]:
test_file_path = 'data/test_a.json'
test_data_list = read_jsonl(test_file_path)

test_df = pd.DataFrame(test_data_list)
test_df.head()

,input,output
0,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock
1,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock
2,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock
3,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock
4,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock


In [11]:
total_eval_count = 0

with open('data/submit.txt','w', encoding="utf-8") as n:
    for index, row in test_df.iterrows():
        input = row["input"]
        output = row["output"]
        
        response, history = base_model.chat(tokenizer=base_tokenizer, query=input)
        optimized_resp = optimize_parameters(response, input)
        
        total_eval_count += 1
        n.write(optimized_resp+'\n')
        if total_eval_count % 10 == 0:
            print("现在是第" + f"{total_eval_count}" + "条数据")

现在是第10条数据
现在是第20条数据
现在是第30条数据
现在是第40条数据
现在是第50条数据
现在是第60条数据
现在是第70条数据
现在是第80条数据
现在是第90条数据
现在是第100条数据
现在是第110条数据
现在是第120条数据
现在是第130条数据
现在是第140条数据
现在是第150条数据
现在是第160条数据
现在是第170条数据
现在是第180条数据
现在是第190条数据
现在是第200条数据
现在是第210条数据
现在是第220条数据
现在是第230条数据
现在是第240条数据
现在是第250条数据
现在是第260条数据
现在是第270条数据
现在是第280条数据
现在是第290条数据
现在是第300条数据
现在是第310条数据
现在是第320条数据
现在是第330条数据
现在是第340条数据
现在是第350条数据
现在是第360条数据
现在是第370条数据
现在是第380条数据
现在是第390条数据
现在是第400条数据
现在是第410条数据
现在是第420条数据
现在是第430条数据
现在是第440条数据
现在是第450条数据
现在是第460条数据
现在是第470条数据
现在是第480条数据
现在是第490条数据
现在是第500条数据
现在是第510条数据
现在是第520条数据
现在是第530条数据
现在是第540条数据
现在是第550条数据
现在是第560条数据
现在是第570条数据
现在是第580条数据
现在是第590条数据
现在是第600条数据
现在是第610条数据
现在是第620条数据
现在是第630条数据
现在是第640条数据
现在是第650条数据
现在是第660条数据
现在是第670条数据
现在是第680条数据
现在是第690条数据
现在是第700条数据
现在是第710条数据
现在是第720条数据
现在是第730条数据
现在是第740条数据
现在是第750条数据
现在是第760条数据
现在是第770条数据
现在是第780条数据
现在是第790条数据
现在是第800条数据
现在是第810条数据
现在是第820条数据
现在是第830条数据
现在是第840条数据
现在是第850条数据
现在是第860条数据
现在是第870条数据
现在是第880条数据
现在是第890条数据
现在是第900条数据
现在是第910条数据
现在是第920条